In [1]:
import dask.bag as db
import dask.array as da
import dask.dataframe as df
import math
import pandas as pd
import numpy as np
import requests

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


Client Scheduler: tcp://127.0.0.1:41609 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 33.68 GB


Helper function that displays all the functions that can be invoked on an object.

In [3]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

In [4]:
def get_split_words(word_bag):    
    split_words = word_bag.str.split()
    stripped_words = split_words.flatten().map(lambda x: x.strip())
    word_array = stripped_words.filter(lambda x: x!= "")
    return word_array

In [5]:
def get_filtered_words(words):
    stopwords = open("/home/dsp_team_gregory/gregory-p1/data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [6]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    split_words = get_split_words(word_bag)
    long_words = split_words.filter(lambda x: len(x) < 3)
#     words_without_punctuations = long_words.map(puntuation_stripper)
    return get_filtered_words(long_words)    
#     return long_words

In [7]:
def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    large_bag = db.concat(unique_words)
    frequencies = large_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], round(math.log((len(words_by_document) + 1)/x[1]), 10)))
    return idf

In [8]:
def get_TF_IDF(tf, idf):
    tf = np.asarray(tf.frequencies().compute())
    tf_idf = {}
    for idf_item in idf: 
        col_name = str(int(idf_item[0], 16))
        tf_idf[col_name] = idf_item[1]
        for i in range(len(tf)):
            tf_item = tf[i]
            if idf_item[0] == tf_item[0]:
                tf_idf[col_name] = float(idf_item[1]) * float(tf_item[1])
                np.delete(tf, i)
                break
    return tf_idf

In [9]:
# set_name = "X_small_train.txt"
set_name = "X_train.txt"
# url = "/home/dsp_team_gregory/gregory-p1/data/train/" + set_name
url = 'https://storage.googleapis.com/uga-dsp/project1/files/' + set_name
file_names = pd.read_csv(url, header=None)[0].to_numpy()

In [10]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

19:43:24


In [11]:
all_file_words = []
for file_name in file_names:
#     file_name = "/home/dsp_team_gregory/gregory-p1/data/train/" + file_name + ".bytes"
    file_name = "https://storage.googleapis.com/uga-dsp/project1/data/bytes/" + file_name + ".bytes"
    all_file_words.append(get_words_for_document(file_name))
idf = get_IDF(all_file_words)

In [12]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

19:43:48


In [ ]:
idf = np.asarray(idf.topk(256, key=0).compute())

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

20:13:14


In [ ]:
cols = np.arange(255, -1, -1).astype(str)
pd_df = pd.DataFrame(columns=cols)
i = 0
for file_words in all_file_words:
    print("working on file number %d" % i)
    i += 1
    tf_idf_bag_array = get_TF_IDF(file_words, idf)
    pd_df = pd_df.append(tf_idf_bag_array, ignore_index=True)

working on file number 0
working on file number 1
working on file number 2
working on file number 3
working on file number 4
working on file number 5
working on file number 6
working on file number 7
working on file number 8
working on file number 9
working on file number 10
working on file number 11
working on file number 12
working on file number 13
working on file number 14
working on file number 15
working on file number 16
working on file number 17
working on file number 18
working on file number 19
working on file number 20
working on file number 21
working on file number 22
working on file number 23
working on file number 24
working on file number 25
working on file number 26
working on file number 27
working on file number 28
working on file number 29
working on file number 30
working on file number 31
working on file number 32
working on file number 33
working on file number 34
working on file number 35
working on file number 36
working on file number 37
working on file number

working on file number 308
working on file number 309
working on file number 310
working on file number 311
working on file number 312
working on file number 313
working on file number 314
working on file number 315
working on file number 316
working on file number 317
working on file number 318
working on file number 319
working on file number 320
working on file number 321
working on file number 322
working on file number 323
working on file number 324
working on file number 325
working on file number 326
working on file number 327
working on file number 328
working on file number 329
working on file number 330
working on file number 331
working on file number 332
working on file number 333
working on file number 334
working on file number 335
working on file number 336
working on file number 337
working on file number 338
working on file number 339
working on file number 340
working on file number 341
working on file number 342
working on file number 343
working on file number 344
w

working on file number 612
working on file number 613
working on file number 614
working on file number 615
working on file number 616
working on file number 617
working on file number 618
working on file number 619
working on file number 620
working on file number 621
working on file number 622
working on file number 623
working on file number 624
working on file number 625
working on file number 626
working on file number 627
working on file number 628
working on file number 629
working on file number 630
working on file number 631
working on file number 632
working on file number 633
working on file number 634
working on file number 635
working on file number 636
working on file number 637
working on file number 638
working on file number 639
working on file number 640
working on file number 641
working on file number 642
working on file number 643
working on file number 644
working on file number 645
working on file number 646
working on file number 647
working on file number 648
w

working on file number 916
working on file number 917
working on file number 918
working on file number 919
working on file number 920
working on file number 921
working on file number 922
working on file number 923
working on file number 924
working on file number 925
working on file number 926
working on file number 927
working on file number 928
working on file number 929
working on file number 930
working on file number 931
working on file number 932
working on file number 933
working on file number 934
working on file number 935
working on file number 936
working on file number 937
working on file number 938
working on file number 939
working on file number 940
working on file number 941
working on file number 942
working on file number 943
working on file number 944
working on file number 945
working on file number 946
working on file number 947
working on file number 948
working on file number 949
working on file number 950
working on file number 951
working on file number 952
w

working on file number 1212
working on file number 1213
working on file number 1214
working on file number 1215
working on file number 1216
working on file number 1217
working on file number 1218
working on file number 1219
working on file number 1220
working on file number 1221
working on file number 1222
working on file number 1223
working on file number 1224
working on file number 1225
working on file number 1226
working on file number 1227
working on file number 1228
working on file number 1229
working on file number 1230
working on file number 1231
working on file number 1232
working on file number 1233
working on file number 1234
working on file number 1235
working on file number 1236
working on file number 1237
working on file number 1238
working on file number 1239
working on file number 1240
working on file number 1241
working on file number 1242
working on file number 1243
working on file number 1244
working on file number 1245
working on file number 1246
working on file numb

...

working on file number 6788
working on file number 6789
working on file number 6790
working on file number 6791
working on file number 6792
working on file number 6793
working on file number 6794
working on file number 6795
working on file number 6796
working on file number 6797
working on file number 6798
working on file number 6799
working on file number 6800
working on file number 6801
working on file number 6802
working on file number 6803
working on file number 6804
working on file number 6805
working on file number 6806
working on file number 6807
working on file number 6808
working on file number 6809
working on file number 6810
working on file number 6811
working on file number 6812
working on file number 6813
working on file number 6814
working on file number 6815
working on file number 6816
working on file number 6817
working on file number 6818
working on file number 6819
working on file number 6820
working on file number 6821
working on file number 6822
working on file numb

working on file number 7081
working on file number 7082
working on file number 7083
working on file number 7084
working on file number 7085
working on file number 7086
working on file number 7087
working on file number 7088
working on file number 7089
working on file number 7090
working on file number 7091
working on file number 7092
working on file number 7093
working on file number 7094
working on file number 7095
working on file number 7096
working on file number 7097
working on file number 7098
working on file number 7099
working on file number 7100
working on file number 7101
working on file number 7102
working on file number 7103
working on file number 7104
working on file number 7105
working on file number 7106
working on file number 7107
working on file number 7108
working on file number 7109
working on file number 7110
working on file number 7111
working on file number 7112
working on file number 7113
working on file number 7114
working on file number 7115
working on file numb

working on file number 7374
working on file number 7375
working on file number 7376
working on file number 7377
working on file number 7378
working on file number 7379
working on file number 7380
working on file number 7381
working on file number 7382
working on file number 7383
working on file number 7384
working on file number 7385
working on file number 7386
working on file number 7387
working on file number 7388
working on file number 7389
working on file number 7390
working on file number 7391
working on file number 7392
working on file number 7393
working on file number 7394
working on file number 7395
working on file number 7396
working on file number 7397
working on file number 7398
working on file number 7399
working on file number 7400
working on file number 7401
working on file number 7402
working on file number 7403
working on file number 7404
working on file number 7405
working on file number 7406
working on file number 7407
working on file number 7408
working on file numb

working on file number 7667
working on file number 7668
working on file number 7669
working on file number 7670
working on file number 7671
working on file number 7672
working on file number 7673
working on file number 7674
working on file number 7675
working on file number 7676
working on file number 7677
working on file number 7678
working on file number 7679
working on file number 7680
working on file number 7681
working on file number 7682
working on file number 7683
working on file number 7684
working on file number 7685
working on file number 7686
working on file number 7687
working on file number 7688
working on file number 7689
working on file number 7690
working on file number 7691
working on file number 7692
working on file number 7693
working on file number 7694
working on file number 7695
working on file number 7696
working on file number 7697
working on file number 7698
working on file number 7699
working on file number 7700
working on file number 7701
working on file numb

working on file number 7960
working on file number 7961
working on file number 7962
working on file number 7963
working on file number 7964
working on file number 7965
working on file number 7966
working on file number 7967
working on file number 7968
working on file number 7969
working on file number 7970
working on file number 7971
working on file number 7972
working on file number 7973
working on file number 7974
working on file number 7975
working on file number 7976
working on file number 7977
working on file number 7978
working on file number 7979
working on file number 7980
working on file number 7981
working on file number 7982
working on file number 7983
working on file number 7984
working on file number 7985
working on file number 7986
working on file number 7987
working on file number 7988
working on file number 7989
working on file number 7990
working on file number 7991
working on file number 7992
working on file number 7993
working on file number 7994
working on file numb

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

22:38:08


In [30]:
csv_name = "/home/dsp_team_gregory/gregory-p1/data/" + set_name + ".csv"
# pd_df.to_csv(csv_name)
csv_name

'/home/dsp_team_gregory/gregory-p1/data/X_train.txt.csv'

In [ ]:
tf_idf_df = df.from_pandas(pd_df, npartitions=1)
final_dataframe = tf_idf_df.compute()

In [ ]:
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

22:38:11


In [ ]:
final_dataframe

,255,254,253,252,251,250,249,248,247,246,...,9,8,7,6,5,4,3,2,1,0
0,11.5027,0.916191,0.62995,0.854301,0.621109,0.572481,1.49177,0.928348,0.544852,0.84767,...,0.915086,2.67563,1.05213,1.28642,1.18254,5.35016,1.66882,7.57598,5.45073,48.7792
1,21.2326,2.2844,1.43894,4.83846,0.792411,0.680788,2.04964,1.90753,1.3461,1.3671,...,1.01676,6.35255,1.45331,1.8158,2.0711,8.31313,6.17572,2.98619,5.84859,75.0415
2,7.88654,3.56751,3.5664,3.55867,3.48794,3.57524,8.02163,3.61724,3.54209,3.52109,...,3.46804,3.56861,3.57524,3.59293,3.61172,3.81728,3.62387,3.51446,6.06189,12.6244
3,7.98269,3.79407,3.46473,3.57967,3.52883,3.40836,7.884,3.51335,3.50562,3.59072,...,3.70897,3.60287,3.67582,3.63492,3.50562,3.78081,3.69239,3.57414,6.20335,12.6764
4,5.0396,0.670842,0.543747,0.769202,0.53601,0.551483,1.22143,0.792411,0.681894,0.658685,...,0.600111,1.3671,0.696261,0.773623,0.904034,1.88322,1.1107,0.926137,4.56769,30.8753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8142,2.57506,2.11752,2.09983,2.10094,2.01142,2.04126,4.6031,2.20483,2.21367,2.16504,...,2.13188,2.16283,2.2623,2.13078,2.13852,2.12525,2.18383,2.04679,2.34739,7.59587
8143,6.45423,2.07552,0.974765,1.28421,1.0245,1.01897,1.69821,1.78707,1.34721,2.06226,...,1.95948,2.65795,2.01805,3.55093,2.35513,4.01953,2.12746,2.33303,3.12654,22.2549
8144,2.10426,1.68208,1.68318,1.65776,1.68981,1.58924,3.60285,1.58482,1.66108,1.61245,...,1.70418,1.58924,1.65113,1.74949,1.97495,1.71523,1.72407,1.63124,1.87217,4.06594
8145,5.18438,3.29121,3.13649,4.25382,2.99724,3.00276,6.90588,3.57967,3.88469,3.50893,...,3.18291,4.19967,3.96537,4.31461,3.43931,3.28016,3.48683,3.92669,4.00737,682.491


In [31]:
%store final_dataframe

Stored 'final_dataframe' (DataFrame)
